In [20]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity
from statistics import mean, mode

In [2]:
def median(arr):
    arr = list(map(int, arr))
    size = len(arr)
    if size % 2:
        return arr[size//2]
    if size != 0:
        return (arr[size//2]+arr[size//2+1])//2
    return 0

In [3]:
def calculateSpatialInfo(img: np.ndarray) -> float:
    """
    Calculate spatial information from a grayscale image using Sobel filters.

    Args:
        img: A grayscale image as a numpy array.

    Returns:
        The spatial information as a float.
    """
    sh = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=1)
    sv = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=1)

    sobel_image = np.sqrt(np.square(sh) + np.square(sv))

    si_mean = np.sum(sobel_image) / (sobel_image.shape[0] * sobel_image.shape[1])
    si_rms = np.sqrt(np.sum(sobel_image ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))
    si_stdev = np.sqrt(np.sum(sobel_image ** 2 - si_mean ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))

    return si_stdev

In [4]:
def getNeighbour(image: np.ndarray, i: int, j: int) -> list:
    neighbour = [[], [], []]
    shape = image.shape
    # , (1, 1), (-1, -1), (1, -1), (-1, 1)
    for di, dj in [(0,0), (0, 1), (0, -1), (1, 0), (-1, 0)]:
        curr_i = i + di
        curr_j = j + dj
        if curr_i < 0 or curr_i >= shape[0]:
            continue
        if curr_j < 0 or curr_j >= shape[1]:
            continue
        for color in [0, 1, 2]:
            neighbour[color].append(image[curr_i][curr_j][color])
    return neighbour


In [5]:
def polishingImage(resized: np.ndarray, func: callable) -> np.ndarray:
    shape = resized.shape
    polished = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            neighbour = getNeighbour(resized, i, j)
            polished[i][j] = [func(neighbour[0]), func(neighbour[1]), func(neighbour[2])]
    return polished

In [6]:
def resizeImage(original: np.ndarray, x_rate: float, y_rate: float, func: callable = None) -> np.ndarray:
    shape = list(original.shape)
    shape[0] = int(shape[0]*x_rate)
    shape[1] = int(shape[1]*y_rate)
    resized = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            orig_i, orig_j = int(i/x_rate),int(j/y_rate)
            if func:
                neighbour = getNeighbour(original, orig_i, orig_j)
                resized[i][j] = [func(neighbour[0]), func(neighbour[1]), func(neighbour[2])]
            else:
                resized[i][j] = [*original[orig_i][orig_j]]
            
    return resized


In [7]:
prefix = 'images/a'
original = {}
generated = {}

In [8]:
original['small'] = cv2.imread(f'{prefix}/small.jpg')
original['big'] = cv2.imread(f'{prefix}/big.jpg')
original['small_gray'] = cv2.imread(f'{prefix}/small.jpg', cv2.IMREAD_GRAYSCALE)
original['big_gray'] = cv2.imread(f'{prefix}/big.jpg', cv2.IMREAD_GRAYSCALE)

In [9]:
# Calculate images spatial info
spatial_info = calculateSpatialInfo(original['small_gray'])
spatial_info

23.20730158646722

In [10]:
## EXPAND
# Obtain resized images the was polished based in the original images
generated['big_ob_mode'] = resizeImage(original['small'], 2, 2, mode)
generated['big_ob_mean'] = resizeImage(original['small'], 2, 2, mean)
generated['big_ob_median'] = resizeImage(original['small'], 2, 2, median)
# Obtain polished images the was polished based in the resized images
generated['big'] = resizeImage(original['small'], 2, 2)
generated['big_rb_mode'] = polishingImage(generated['big'], mode)
generated['big_rb_mean'] = polishingImage(generated['big'], mean)
generated['big_rb_median'] = polishingImage(generated['big'], median)
# Using bilinear and bicubic methods
shape = original['big'].shape[1::-1]
generated['big_bilinear'] = cv2.resize(original['small'], shape, interpolation=cv2.INTER_LINEAR)
generated['big_bicubic'] = cv2.resize(original['small'], shape, interpolation=cv2.INTER_CUBIC)

In [11]:
## REDUCE
# Obtain resized images the was polished based in the original images
generated['small_ob_mode'] = resizeImage(original['big'], .5, .5, mode)
generated['small_ob_mean'] = resizeImage(original['big'], .5, .5, mean)
generated['small_ob_median'] = resizeImage(original['big'], .5, .5, median)
# Obtain polished images the was polished based in the resized images
generated['small'] = resizeImage(original['big'], .5, .5)
generated['small_rb_mode'] = polishingImage(generated['small'], mode)
generated['small_rb_mean'] = polishingImage(generated['small'], mean)
generated['small_rb_median'] = polishingImage(generated['small'], median)
# Using bilinear and bicubic methods
shape = original['small'].shape[1::-1]
generated['small_bilinear'] = cv2.resize(original['small'], shape, interpolation=cv2.INTER_LINEAR)
generated['small_bicubic'] = cv2.resize(original['small'], shape, interpolation=cv2.INTER_CUBIC)

In [12]:
for image in generated:
    cv2.imwrite(f'{prefix}/generated/{image}.jpg', generated[image])

In [34]:
results_big = []
results_small = []
for image in generated:
    generated_gray = cv2.imread(f'{prefix}/generated/{image}.jpg', cv2.IMREAD_GRAYSCALE)
    if 'small' in image:
        ssim = structural_similarity(original['small_gray'], generated_gray, full=True)[0]
        psnr = cv2.PSNR(original['small_gray'], generated_gray)
        results_small.append((psnr, ssim, image))
    else:
        ssim = structural_similarity(original['big_gray'], generated_gray, full=True)[0]
        psnr = cv2.PSNR(original['big_gray'], generated_gray)
        results_big.append((psnr, ssim, image))

In [39]:
results_big.sort(reverse=True)
for psnr, ssim, name in results_big:
    print(f'{name:13s} - PSNR {psnr:.4f} - SSIM  {ssim:.4f}' )

big_bicubic   - PSNR 27.5338 - SSIM  0.7771
big_rb_mean   - PSNR 27.2531 - SSIM  0.7627
big_bilinear  - PSNR 27.2291 - SSIM  0.7610
big_rb_mode   - PSNR 26.4775 - SSIM  0.7436
big           - PSNR 26.4775 - SSIM  0.7436
big_ob_mode   - PSNR 26.4291 - SSIM  0.7375
big_ob_mean   - PSNR 25.7721 - SSIM  0.6926
big_rb_median - PSNR 24.8119 - SSIM  0.6790
big_ob_median - PSNR 22.5210 - SSIM  0.5473


In [40]:
results_small.sort(reverse=True)
for psnr, ssim, name in results_small:
    print(f'{name:15s} - PSNR {psnr:.4f} - SSIM  {ssim:.4f}')

small_bilinear  - PSNR 51.3656 - SSIM  0.9989
small_bicubic   - PSNR 51.3656 - SSIM  0.9989
small_ob_mean   - PSNR 29.8728 - SSIM  0.8802
small_rb_mean   - PSNR 28.5102 - SSIM  0.8582
small_rb_mode   - PSNR 26.7543 - SSIM  0.8066
small_ob_mode   - PSNR 26.6614 - SSIM  0.8026
small           - PSNR 26.4441 - SSIM  0.7925
small_ob_median - PSNR 23.6439 - SSIM  0.6730
small_rb_median - PSNR 21.6035 - SSIM  0.5328
